In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

class PoisonedDataset(Dataset):
    def __init__(self, poison_list_path, poison_data_dir, transform=None):
        """
        Initialize the poisoned dataset.
        :param poison_list_path: Path to poison_list.txt
        :param poison_data_dir: Directory containing poisoned images
        :param transform: Optional transformations for the images
        """
        self.transform = transform
        self.poison_data_dir = poison_data_dir
        
        # Load poison list
        self.poison_indices_and_labels = []
        with open(poison_list_path, "r") as f:
            for line in f:
                index, label = map(int, line.strip().split())
                self.poison_indices_and_labels.append((index, label))
        
        # Sort poison list
        self.poison_indices_and_labels.sort(key=lambda x: x[0])

        # Get valid image files only
        self.image_files = [
            f for f in sorted(os.listdir(poison_data_dir)) 
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        
        # Check for mismatches
        if len(self.image_files) < len(self.poison_indices_and_labels):
            print(f"Warning: Fewer image files ({len(self.image_files)}) than entries in poison_list.txt ({len(self.poison_indices_and_labels)})")
            self.poison_indices_and_labels = self.poison_indices_and_labels[:len(self.image_files)]
        
        elif len(self.image_files) > len(self.poison_indices_and_labels):
            print(f"Warning: More image files ({len(self.image_files)}) than entries in poison_list.txt ({len(self.poison_indices_and_labels)})")
            self.image_files = self.image_files[:len(self.poison_indices_and_labels)]

    def __len__(self):
        return len(self.poison_indices_and_labels)
    
    def __getitem__(self, idx):
        """
        Get a single image-label pair.
        """
        label = self.poison_indices_and_labels[idx][1]
        image_file = self.image_files[idx]
        image_path = os.path.join(self.poison_data_dir, image_file)
        
        # Load image
        image = Image.open(image_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [2]:
# Paths
poison_list_path = "/home/j597s263/scratch/j597s263/Hidden-Trigger-Backdoor-Attacks/CIFAR10_data_list/poison_generation/poison_list.txt"
poison_data_dir = "/home/j597s263/scratch/j597s263/Hidden-Trigger-Backdoor-Attacks/poison_data/6002/rand_loc_True/eps_8/patch_size_30/trigger_14"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

# Create poison dataset
poisoned_dataset = PoisonedDataset(poison_list_path, poison_data_dir, transform=transform)

# Test dataset
print(f"Number of images in poison dataset: {len(poisoned_dataset)}")

# Example: Fetch a single sample
image, label = poisoned_dataset[0]
print(f"Image shape: {image.shape}, Label: {label}")


Number of images in poison dataset: 1000
Image shape: torch.Size([3, 224, 224]), Label: 7


In [3]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset

class CleanCIFARDataset(Dataset):
    def __init__(self, root, poisoned_indices, train=True, transform=None):
        """
        Initialize the clean CIFAR dataset, excluding poisoned indices.
        :param root: Path to CIFAR data
        :param poisoned_indices: Set of poisoned indices
        :param train: Whether to use training or test data
        :param transform: Optional transformations for the images
        """
        self.transform = transform
        self.cifar = datasets.CIFAR10(root=root, train=train, download=False)
        self.poisoned_indices = poisoned_indices

        # Filter out poisoned indices
        self.filtered_data = [
            (self.cifar.data[idx], self.cifar.targets[idx])
            for idx in range(len(self.cifar))
            if idx not in self.poisoned_indices
        ]

    def __len__(self):
        return len(self.filtered_data)
    
    def __getitem__(self, idx):
        """
        Get a single image-label pair.
        """
        image, label = self.filtered_data[idx]
        image = Image.fromarray(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# Paths
cifar_root = "/home/j597s263/scratch/j597s263/Datasets/cifar10"
# Get indices of poisoned data
poisoned_indices = set()
for idx, _ in poisoned_dataset.poison_indices_and_labels:
    poisoned_indices.add(idx)

# Create clean CIFAR-10 dataset
clean_dataset = CleanCIFARDataset(cifar_root, poisoned_indices, train=True, transform=transform)

# Print stats
print(f"Number of samples in clean CIFAR-10 dataset: {len(clean_dataset)}")

# Test dataset
image, label = clean_dataset[0]
print(f"Image shape: {image.shape}, Label: {label}")


Number of samples in clean CIFAR-10 dataset: 49000
Image shape: torch.Size([3, 224, 224]), Label: 6


In [7]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset
from torch.utils.data import DataLoader, ConcatDataset, Subset
import random

# Debugging Step: Check dataset sizes
clean_test_size = int(len(clean_dataset) * 0.1)
poison_test_size = int(len(poisoned_dataset) * 0.1)

print(f"Expected clean test size: {clean_test_size}")
print(f"Expected poison test size: {poison_test_size}")

# Generate random indices for 10% splits
clean_test_indices = random.sample(range(len(clean_dataset)), clean_test_size)
poison_test_indices = random.sample(range(len(poisoned_dataset)), poison_test_size)

# Create Subsets for testing
clean_test_dataset = Subset(clean_dataset, clean_test_indices)
poison_test_dataset = Subset(poisoned_dataset, poison_test_indices)

# Remove test indices from original datasets for training
clean_train_indices = list(set(range(len(clean_dataset))) - set(clean_test_indices))
poison_train_indices = list(set(range(len(poisoned_dataset))) - set(poison_test_indices))

clean_train_dataset = Subset(clean_dataset, clean_train_indices)
poison_train_dataset = Subset(poisoned_dataset, poison_train_indices)

# Debugging Step: Print dataset lengths
print(f"Clean train dataset size: {len(clean_train_dataset)}")
print(f"Clean test dataset size: {len(clean_test_dataset)}")
print(f"Poison train dataset size: {len(poison_train_dataset)}")
print(f"Poison test dataset size: {len(poison_test_dataset)}")

# Combine the remaining clean and poisoned datasets for training
combined_train_dataset = ConcatDataset([clean_train_dataset, poison_train_dataset])

# Create DataLoaders
train_loader = DataLoader(combined_train_dataset, batch_size=128, shuffle=True)
test_loader_clean = DataLoader(clean_test_dataset, batch_size=128, shuffle=False)
test_loader_poison = DataLoader(poison_test_dataset, batch_size=128, shuffle=False)

# Debugging Step: Print DataLoader lengths
print(f"Train loader batches: {len(train_loader)}")
print(f"Clean test loader batches: {len(test_loader_clean)}")
print(f"Poison test loader batches: {len(test_loader_poison)}")


Expected clean test size: 4900
Expected poison test size: 100
Clean train dataset size: 44100
Clean test dataset size: 4900
Poison train dataset size: 900
Poison test dataset size: 100
Train loader batches: 352
Clean test loader batches: 39
Poison test loader batches: 1


In [8]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [9]:
import torch
# Load the entire model
model = torch.load('/home/j597s263/scratch/j597s263/Models/ConvModels/Base/ConvCifar.mod', weights_only=False, map_location="cuda:0")

# Move the model to the appropriate device
model = model.to('cuda')

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [10]:
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler

device = 'cuda'
epochs = 15
learning_rate = 0.001
opt_eps = 1e-3
clip_grad = 1.0

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(train_loader),
    epochs=epochs
)

scaler = GradScaler()

def evaluate_model(model, data_loader, device, dataset_type="dataset"):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on {dataset_type}: {accuracy:.2f}%")
    return accuracy

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Training Loss: {running_loss / len(train_loader):.4f}")

    poisoned_test_accuracy = evaluate_model(model, test_loader_poison, device, dataset_type="poisoned test dataset")

    clean_test_accuracy = evaluate_model(model, test_loader_clean, device, dataset_type="clean test dataset")

    print(f"Epoch [{epoch + 1}/{epochs}] - Poisoned Test Accuracy: {poisoned_test_accuracy:.2f}%, Clean Test Accuracy: {clean_test_accuracy:.2f}%")


/tmp/ipykernel_289160/2124902200.py:20: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_289160/2124902200.py:46: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/15], Training Loss: 0.1196
Accuracy on poisoned test dataset: 9.00%
Accuracy on clean test dataset: 99.18%
Epoch [1/15] - Poisoned Test Accuracy: 9.00%, Clean Test Accuracy: 99.18%
Epoch [2/15], Training Loss: 0.0941
Accuracy on poisoned test dataset: 10.00%
Accuracy on clean test dataset: 99.12%
Epoch [2/15] - Poisoned Test Accuracy: 10.00%, Clean Test Accuracy: 99.12%
Epoch [3/15], Training Loss: 0.0675
Accuracy on poisoned test dataset: 15.00%
Accuracy on clean test dataset: 99.20%
Epoch [3/15] - Poisoned Test Accuracy: 15.00%, Clean Test Accuracy: 99.20%
Epoch [4/15], Training Loss: 0.0548
Accuracy on poisoned test dataset: 7.00%
Accuracy on clean test dataset: 99.16%
Epoch [4/15] - Poisoned Test Accuracy: 7.00%, Clean Test Accuracy: 99.16%
Epoch [5/15], Training Loss: 0.0479
Accuracy on poisoned test dataset: 18.00%
Accuracy on clean test dataset: 99.16%
Epoch [5/15] - Poisoned Test Accuracy: 18.00%, Clean Test Accuracy: 99.16%
Epoch [6/15], Training Loss: 0.0431
Accuracy